<a href="https://colab.research.google.com/github/coppermilk/colab/blob/main/WhatsApp_Explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install the required libraries.

In [ ]:
pip install urlextract


2. Download chat.

In [6]:
import regex
import pandas as pd
import numpy as np
from urlextract import URLExtract
from IPython.display import HTML
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

3. Start parser.

In [ ]:
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = regex.match(pattern, s)
    if result:
        return True
    return False

def find_author(s):
    s = s.split(":")
    if len(s) >= 2:
        return True
    else:
        return False

def try_join(l):
    try:
        return ','.join(map(str, l))
    except TypeError:
        return np.nan

def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    # date, time = dateTime.split(", ")
    # dateTime = date + " " + time
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        if('location' in splitmessage):
            #author += splitmessage[1]
            message = " ".join(splitmessage[1:])
        else:
            message = " ".join(splitmessage[1:])
        message.replace('\n', '')
    else:
        author = None
    return dateTime, author, message

data = []
with open(fn, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    dateTime, author = None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([dateTime, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            dateTime, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

df = pd.DataFrame(data, columns=['DateTime', 'Author', 'Message'])

# Convert datatime
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Extract links
extractor = URLExtract()
df['Url'] = df.Message.apply(extractor.find_urls)
df['Url2'] = [try_join(l) for l in df['Url']]

author_list = df.Author.unique();
s1 = pd.Series(author_list, range(0, author_list.size))
s1

4.1 DateTime, Author, Message.

In [ ]:
id = 20
start_from = '2022-11-01'
mask = (df.Author == author_list[id]) & (df.DateTime >= pd.Timestamp(start_from)) & df.Url2
new_df = df[['DateTime','Author', 'Url2']].copy()
HTML(new_df[mask].to_html(render_links=True, escape=False))



4.2 DateTime, Authohor, Message, Url.

In [ ]:
id = 20
start_from = '2022-11-01'
mask = (df.Author == author_list[id]) & (df.DateTime >= pd.Timestamp(start_from))
df2 = df[['DateTime','Author','Message',  'Url2']].copy()
HTML(df2[mask].to_html(render_links=True, escape=False))

